In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
DATASET_PUBMED_RCT = './gdrive/Shareddrives/DATASETS/PUBMED_RCT/'
OUTPUT_MODEL_DIR = './gdrive/Shareddrives/MODELS/'

In [3]:
!pip install transformers -q
!pip install sentence_transformers -q

     |████████████████████████████████| 4.7 MB 13.0 MB/s 
     |████████████████████████████████| 6.6 MB 28.7 MB/s 
     |████████████████████████████████| 101 kB 9.8 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
     |████████████████████████████████| 85 kB 3.6 MB/s 
     |████████████████████████████████| 1.3 MB 30.9 MB/s 


In [4]:
import pandas as pd

In [5]:
df_train = pd.read_parquet(DATASET_PUBMED_RCT + 'train.parquet')
df_dev = pd.read_parquet(DATASET_PUBMED_RCT + 'df_dev_triplets.parquet')
df_test = pd.read_parquet(DATASET_PUBMED_RCT + 'df_test_triplets.parquet')

In [6]:
df_train.columns

Index(['pmid', 'label', 'sentence', 'label_id'], dtype='object')

In [7]:
df_dev.columns

Index(['anchor', 'positive', 'negative'], dtype='object')

In [8]:
# same text but with different labels
print(f'{len(df_train.index)}')
mask = df_train.groupby('sentence')['label'].transform('nunique') > 1
df_train = df_train[~mask].copy()
print(f'{len(df_train.index)}')

180040
179892


In [9]:
from sentence_transformers import InputExample
from tqdm import tqdm

train_set = []
guid = 1
for idx, row in tqdm(df_train.iterrows(), total=len(df_train.index)):
    train_set.append(InputExample(
        guid=guid,
        texts=[row['sentence']],
        label=row['label_id']
    ))
    guid += 1
len(train_set)

100%|██████████| 179892/179892 [00:23<00:00, 7705.26it/s] 


179892

In [10]:
dev_set = []

guid = 1
for idx, row in tqdm(df_dev.iterrows(), total=len(df_dev.index)):
    dev_set.append(InputExample(
        guid=guid,
        texts=[row['anchor'], row['positive'], row['negative']],
    ))
    guid += 1
len(dev_set)

100%|██████████| 30212/30212 [00:02<00:00, 11405.99it/s]


30212

In [11]:
test_set = []

guid = 1
for idx, row in tqdm(df_test.iterrows(), total=len(df_test.index)):
    test_set.append(InputExample(
        guid=guid,
        texts=[row['anchor'], row['positive'], row['negative']],
    ))
    guid += 1
len(test_set)

100%|██████████| 30122/30122 [00:02<00:00, 10463.30it/s]


30122

In [12]:
import logging
from sentence_transformers import LoggingHandler

logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [13]:
from datetime import datetime
model_name = 'allenai/scibert_scivocab_uncased'
model_file_name = model_name.split('/')[-1] + '_PubMedRCT_TripletAll'

train_batch_size = 16
output_path = (
    OUTPUT_MODEL_DIR
    + model_file_name
    + "-"
    + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)
output_path

'./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16'

In [14]:
from sentence_transformers import models, SentenceTransformer

bert = models.Transformer(model_name)
pooler = models.Pooling(
    bert.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)

model = SentenceTransformer(modules=[bert, pooler])

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2022-08-08 15:45:36 - Use pytorch device: cuda


In [15]:
from sentence_transformers import datasets
from torch.utils.data import DataLoader

loader = DataLoader(train_set, shuffle=True, batch_size=train_batch_size)

In [16]:
from sentence_transformers import losses

train_loss = losses.BatchAllTripletLoss(model=model)

In [17]:
from sentence_transformers.evaluation import TripletEvaluator

dev_evaluator = TripletEvaluator.from_input_examples(
    dev_set, write_csv=True, show_progress_bar=True, name='pubmed-rct-dev'
)

In [18]:
logging.info("Performance before fine-tuning:")
dev_evaluator(model)

2022-08-08 15:45:36 - Performance before fine-tuning:
2022-08-08 15:45:36 - TripletEvaluator: Evaluating the model on pubmed-rct-dev dataset:


Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

2022-08-08 15:48:44 - Accuracy Cosine Distance:   	63.08
2022-08-08 15:48:44 - Accuracy Manhattan Distance:	62.20
2022-08-08 15:48:44 - Accuracy Euclidean Distance:	62.13



0.6308420495167483

In [19]:
logging.info("Evaluating model on test set")
test_evaluator = TripletEvaluator.from_input_examples(
    test_set, write_csv=True, show_progress_bar=True, name='pubmed-rct-test'
)
test_evaluator(model)

2022-08-08 15:48:44 - Evaluating model on test set
2022-08-08 15:48:44 - TripletEvaluator: Evaluating the model on pubmed-rct-test dataset:


Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

2022-08-08 15:51:40 - Accuracy Cosine Distance:   	61.99
2022-08-08 15:51:40 - Accuracy Manhattan Distance:	61.76
2022-08-08 15:51:40 - Accuracy Euclidean Distance:	61.37



0.6198791580904323

In [20]:
num_epochs = 20

warmup_steps = int(len(loader) * num_epochs  * 0.1)  # 10% of train data

In [21]:
model_output_path = output_path + '/model'
checkpoint_output_path = output_path + '/checkpoint'

print(model_output_path)
print(checkpoint_output_path)

./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/model
./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint


In [22]:
%%time
model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_output_path,
    show_progress_bar=True,
    evaluator=dev_evaluator,
    save_best_model=True,
    checkpoint_save_total_limit=1,
    checkpoint_path=checkpoint_output_path    
)  

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/11244 [00:00<?, ?it/s]

2022-08-08 15:53:06 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/500
2022-08-08 15:54:32 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/1000
2022-08-08 15:55:56 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/1500
2022-08-08 15:57:21 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/2000
2022-08-08 15:58:44 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/2500
2022-08-08 16:00:08 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/3000
2022-08-08 16:01:32 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

...

2022-08-09 02:40:44 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/214000
2022-08-09 02:42:08 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/214500
2022-08-09 02:43:31 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/215000
2022-08-09 02:44:55 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/215500
2022-08-09 02:46:19 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/216000
2022-08-09 02:47:42 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/216500
2022-08-09 02:49:06 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_Triple

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

Batches:   0%|          | 0/1889 [00:00<?, ?it/s]

2022-08-09 03:13:43 - Accuracy Cosine Distance:   	93.49
2022-08-09 03:13:43 - Accuracy Manhattan Distance:	93.51
2022-08-09 03:13:43 - Accuracy Euclidean Distance:	93.51

2022-08-09 03:13:43 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16/checkpoint/224880
CPU times: user 9h 41min 47s, sys: 1h 21min 58s, total: 11h 3min 45s
Wall time: 11h 22min 4s


In [23]:
logging.info("Evaluating model on test set")
test_evaluator = TripletEvaluator.from_input_examples(
    test_set, write_csv=True, show_progress_bar=True, name='pubmed-rct-test'
)
model.evaluate(test_evaluator)

2022-08-09 03:13:45 - Evaluating model on test set
2022-08-09 03:13:45 - TripletEvaluator: Evaluating the model on pubmed-rct-test dataset:


Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

Batches:   0%|          | 0/1883 [00:00<?, ?it/s]

2022-08-09 03:16:38 - Accuracy Cosine Distance:   	92.78
2022-08-09 03:16:38 - Accuracy Manhattan Distance:	92.89
2022-08-09 03:16:38 - Accuracy Euclidean Distance:	92.80



0.9288891839851271

In [24]:
model.save(output_path + '_last')

2022-08-09 03:16:38 - Save model to ./gdrive/Shareddrives/MODELS/scibert_scivocab_uncased_PubMedRCT_TripletAll-2022-08-08_15-45-16_last
